In [ ]:
%load_ext autoreload
%autoreload 2
%aimport
# %reload_ext autoreload

import sys 
from os.path import abspath
paths = [abspath('../..'), "/home/imendoza/alcca/nbody-relaxed/packages/minnow"]

for path in paths: 
    if path not in sys.path: 
        sys.path.insert(0, path)

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
import re 
from astropy.table import Table
import astropy.table

from relaxed import utils 
from relaxed import halo_parameters, halo_catalogs 

# Load catalog

In [ ]:
# load catalog we know and love
fpath = Path('/home/imendoza/alcca/nbody-relaxed/data/Bolshoi/minh/hlist_1.00035.minh')
name = 'BolshoiP'

# this command just sets up the attributes and filters. Catalog is loaded in the next cell from the *.minh file.
hcat = halo_catalogs.HaloCatalog(name, fpath)

In [ ]:
# Load catalog from minh and apply default filters on-the-fly.  
hcat.load_cat_minh()
print('rows in catalog: ', len(hcat.cat))

# Add additional features to the catalog. 

In [ ]:
# Once we loaded the base catalog we are ready to add progenitor and subhalo information. First load packages. 
from relaxed import subhaloes, progenitors

In [ ]:
# The progenitor file requires a csv with the summarized information. In a future notebook I will explain how to obtain it. 
# For now we use the submodules subhalos.catalog and progenitors.catalog which contain functions that interact with the halo_catalog
# in this example they add additional information. 
from pminh import minh

prog_file = '/home/imendoza/alcca/nbody-relaxed/data/trees_bolshoi/summarize.csv'
with minh.open(fpath) as mcat: 
    hcat = subhaloes.catalog.add_subhalo_info(hcat, mcat)
hcat = progenitors.catalog.add_progenitor_info(hcat, prog_file)

In [ ]:
# Now we can take a look at the substructure fraction, for example, caculated in the add_subhalo_info function. 
plt.hist(hcat.cat['f_sub'], bins=50, range=(0,1));

# Produce some plots 

In [ ]:
# All the code necessary to produce plots for the paper will be stored in /figures/code. Note this is not contained inside the main source code directory /relaxed.
# All plots*.py file simply contain functions you can run on a list of HaloCatalogs 'hcats'. 
from figures.code import plots1
from matplotlib.backends.backend_pdf import PdfPages

from relaxed.halo_filters import HaloFilter, get_bound_filter, get_relaxed_filters

In [ ]:
# use halo_filters to create additional halo_catalogs for analysis. 
hcat_m11 = HaloFilter(get_bound_filter('mvir', high=11.22, modifier=np.log10), 'M11')(hcat)
hcat_m12 = HaloFilter(get_bound_filter('mvir', 12,12.2, modifier=np.log10), 'M12')(hcat)
hcat_m13 = HaloFilter(get_bound_filter('mvir',13,14,modifier=np.log10), 'M13')(hcat)

power_cat = HaloFilter(get_relaxed_filters('power2011'), 'power2011')(hcat)
neto_cat = HaloFilter(get_relaxed_filters('neto2007'), 'neto2007')(hcat)
    

In [ ]:
# make sure length of catalogs make sense
print(len(hcat))

print(len(hcat_m11), len(hcat_m12), len(hcat_m13))

# 500 haloes are so > 13.75 
print(len(power_cat), len(neto_cat))

## Plots comparing all the catalog vs relaxed catalogs

In [ ]:
out='test1.pdf'
hcats = [hcat, power_cat, neto_cat]
pdf_path = utils.figure_path.joinpath(out)
with PdfPages(pdf_path) as pdf:
    plots1.plot_mvir_histogram(hcats, pdf)
    plots1.plot_scatter_relaxed_and_mass(hcats, pdf)


## Plots comparing mass binned (decades) versions of catalog

In [ ]:
out='test2.pdf'
hcats = [hcat_m11, hcat_m12, hcat_m13]
pdf_path = utils.figure_path.joinpath(out)
with PdfPages(pdf_path) as pdf:
#     plots1.plot_correlation_matrix_basic(hcats, pdf=pdf)
#     plots1.plot_mean_centered_hists(hcats, pdf)
    plots1.plot_pair_relaxed_correlation(hcats, pdf)